This is a classification model, I am using a fertility dataset collected by WHO in 2010 from 100 volunteers.  I am classify if there was Normal Sperm count or abnormal sperm count giving the features.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

For the text import, it has the following information for columns.

Season in which the analysis was performed. 	1) winter, 2) spring, 3) Summer, 4) fall. 	(-1, -0.33, 0.33, 1) 

Age at the time of analysis. 	within 18-36 	(0, 1) 

Childish diseases (ie , chicken pox, measles, mumps, polio)	1) yes, 2) no. 	(0, 1) 

Accident or serious trauma 	1) yes, 2) no. 	(0, 1) 

Surgical intervention 	1) yes, 2) no. 	(0, 1) 

High fevers in the last year 	1) less than three months ago, 2) more than three months ago, 3) no. 	(-1, 0, 1) 

Frequency of alcohol consumption 	1) several times a day, 2) every day, 3) several times a week, 4) once a week, 5) hardly ever or never 	(0, 1) 

Smoking habit 	1) never, 2) occasional 3) daily. 	(-1, 0, 1) 

Number of hours spent sitting per day 	less or more than 16	(0, 1) 

Output: Diagnosis	normal (N), altered (O)	

In [2]:
Column_names=['season','age','child_disease','accident','surgery','fever','drinking','smoking','idle','diagnosis']
fertility=pd.read_csv("fertility_Diagnosis.txt", delimiter=",", names=Column_names)
fertility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         100 non-null    float64
 1   age            100 non-null    float64
 2   child_disease  100 non-null    int64  
 3   accident       100 non-null    int64  
 4   surgery        100 non-null    int64  
 5   fever          100 non-null    int64  
 6   drinking       100 non-null    float64
 7   smoking        100 non-null    int64  
 8   idle           100 non-null    float64
 9   diagnosis      100 non-null    object 
dtypes: float64(4), int64(5), object(1)
memory usage: 7.9+ KB


There is no Null value so no need to change.

Changing the output Int object (1 being Normal, and 0 being Abnormal)

In [3]:
fertility['diagnosis']=fertility['diagnosis'].apply(lambda x: 1 if x=='N' else 0)

Spliting Data info Training and Testing

In [4]:
fX = fertility.iloc[:,:-1]
fy = fertility['diagnosis']
fX_train, fX_test, fy_train, fy_test = train_test_split(fX, fy, test_size=0.35, random_state=5)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
scaler = StandardScaler()
scaler.fit(fX_train)
fX_train=scaler.transform(fX_train)
fX_test=scaler.transform(fX_test)
model_fert_KNN = KNeighborsClassifier(n_neighbors=3)
model_fert_KNN.fit(fX_train, fy_train)

KNeighborsClassifier(n_neighbors=3)

In [6]:
fy_pred=model_fert_KNN.predict(fX_test)
print(confusion_matrix(fy_test, fy_pred))
print(classification_report(fy_test, fy_pred))

[[ 0  4]
 [ 0 31]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.89      1.00      0.94        31

    accuracy                           0.89        35
   macro avg       0.44      0.50      0.47        35
weighted avg       0.78      0.89      0.83        35


C:\Users\Xi\miniconda3\envs\ml_lab1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Xi\miniconda3\envs\ml_lab1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Xi\miniconda3\envs\ml_lab1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finding the sample bias and upsample Abnormal data sample, then added to data.

In [7]:
from sklearn.utils import resample
fertility_majority=fertility[fertility['diagnosis']==1]
fertility_minority=fertility[fertility['diagnosis']==0]

# Upsample minority class
df_minority_upsampled = resample(fertility_minority, replace=True, n_samples=len(fertility_majority), random_state=2)

# Combine majority class with upsampled minority class
fertility_upsampled = pd.concat([fertility_majority, df_minority_upsampled])

# Display new class counts
print(fertility_upsampled.diagnosis.value_counts())

diagnosis
1    88
0    88
Name: count, dtype: int64


Resample the training and testing data with the upscaled data.

In [8]:
fX_up = fertility_upsampled.iloc[:,:-1]
fy_up = fertility_upsampled['diagnosis']
fX_up_train, fX_up_test, fy_up_train, fy_up_test = train_test_split(fX_up, fy_up, test_size=0.35, random_state=5)

Scaler the training data and train using KNN

In [9]:
scaler = StandardScaler()
scaler.fit(fX_up_train)
fX_up_train=scaler.transform(fX_up_train)
fX_up_test=scaler.transform(fX_up_test)
model_fert_KNN = KNeighborsClassifier(n_neighbors=5)
model_fert_KNN.fit(fX_up_train, fy_up_train)

KNeighborsClassifier()

Doing Random Forest data using Original Data and finding accuracy information

In [11]:
from sklearn.ensemble import RandomForestClassifier

# create a RandomForestClassifier
model_fert_clf = RandomForestClassifier(n_estimators=100)

# train the model
model_fert_clf.fit(fX_train, fy_train)

# make predictions
fy_clf_pred = model_fert_clf.predict(fX_test)

# print the model accuracy
print("Accuracy:", sum(fy_clf_pred == fy_test) / len(fy_test))
print(confusion_matrix(fy_test, fy_clf_pred))
print(classification_report(fy_test, fy_clf_pred))

Accuracy: 0.8857142857142857
[[ 0  4]
 [ 0 31]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.89      1.00      0.94        31

    accuracy                           0.89        35
   macro avg       0.44      0.50      0.47        35
weighted avg       0.78      0.89      0.83        35


C:\Users\Xi\miniconda3\envs\ml_lab1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Xi\miniconda3\envs\ml_lab1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Xi\miniconda3\envs\ml_lab1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Since the model is obviously having a bias result, using the upsampled training and testing data to train using RandomForest again

In [12]:
model_fert_clf = RandomForestClassifier(n_estimators=100)

# train the model
model_fert_clf.fit(fX_up_train, fy_up_train)

# make predictions
fy_clf_pred = model_fert_clf.predict(fX_test)

# print the model accuracy
print("Accuracy:", sum(fy_clf_pred == fy_test) / len(fy_test))
print(confusion_matrix(fy_test, fy_clf_pred))
print(classification_report(fy_test, fy_clf_pred))

Accuracy: 0.9714285714285714
[[ 4  0]
 [ 1 30]]
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.97      0.98        31

    accuracy                           0.97        35
   macro avg       0.90      0.98      0.94        35
weighted avg       0.98      0.97      0.97        35


It has very respected accuracy of 0.97 percent.  We will then pickle the model and save it.

In [14]:
from joblib import dump, load

# Save the model as a pickle file
dump(model_fert_clf, 'model_fert_clf.pkl')

['model_fert_clf.pkl']

Citation: Gil,David and Girela,Jose. (2013). Fertility. UCI Machine Learning Repository. https://doi.org/10.24432/C5Z01Z.